In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
data = pd.read_csv('/content/drive/MyDrive/geekstest1.csv',encoding = "ISO-8859-1")  
data = data.drop(['url','rating'],axis=1)
data

,title,content,tags
0,Python | Thresholding techniques using OpenCV ...,Prerequisite: Simple Thresholding using OpenC...,"{'Python', 'OpenCV', 'Image-Processing'}"
1,Generation of n numbers with given set of factors,"Given an array of k numbers factor[], the task...","{'prime-factor', 'Mathematical'}"
2,C | Operators | Question 17,Which of the following can have different mean...,"{'C', 'C-Operators', 'Operators', 'C Quiz'}"
3,SymPy | Subset.size() in Python,Subset.size() : size() is a sympy Python libra...,{'Python'}
4,Python PIL | ImageFont.truetype(),PIL is the Python Imaging Library which provid...,"{'Python', 'Python-pil'}"
...,...,...,...
24484,Importance of Hashcode method in Java,Prerequisite: Equals() and hashCode() methods ...,"{'Java', 'java-hashset', 'Java-HashMap'}"
24485,Ruby | Queue close() function,The close() is an inbuilt function in Ruby cl...,"{'Ruby Collections', 'Ruby'}"
24486,Scala | Field Overriding,"In any object-oriented programming language, O...","{'Scala', 'Scala-OOPS'}"
24487,Sum of minimum elements of all possible sub-ar...,"Given an array arr[], the task is to find the ...","{'C++ Programs', 'Data Structures', 'Stack', '..."


In [ ]:
corpus = []
titletoind_dict = {} #mapping of conversation id with corpus index
j = 0
for sub,body in zip(data['title'],data['content']):
    if body is None or body is '':
        corpus += [(sub)]
    else:
        corpus += [(sub+" "+body)]
    titletoind_dict[j] = sub
    j += 1
len(corpus)

24489

In [ ]:
!pip install requests
!pip install spacy
!pip install MySQL
!pip install mysql-connector-python
!pip install MySQL-python
!python -m spacy download en
!pip install --user scikit-learn

     |████████████████████████████████| 92kB 3.6MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.0.3-cp37-cp37m-linux_x86_64.whl size=100117 sha256=5dc4451787d409e6f2de4c1cb54a0aae35a7b6d371cef3aba2de34b39d59047f
  Stored in directory: /root/.cache/pip/wheels/75/ca/e8/ad4e7ce3df18bcd91c7d84dd28c7c08db491a2a2360efed363
Successfully built mysqlclient
     |████████████████████████████████| 25.4MB 129kB/s 
     |████████████████████████████████| 112kB 4.2MB/s 
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
import spacy 
import string,re
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer(language='english')

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[\!\"\#\$\%\&\(\)\*\+\,\-\/\:\;\<\=\>\?\@\^\`\{\}\|\~\\\]')
REPLACE_BY_NULL_RE = re.compile('[\']')
ver = '[0-9]+\.[0-9]+\.*[0-9]*' #regex that identifies version number 
dot = re.compile('[/.]') #identifies '.' in tokens
fs = '[a-z]*\.[a-z]*' #regex that identifies words having '.'
cr_pattern = re.compile(pattern = "["u'\N{COPYRIGHT SIGN} \N{TRADE MARK SIGN} \N{REGISTERED SIGN}'"]+", flags = re.UNICODE)#remove copyright symbols
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)
def preprocess(body):
    if body is None:
        return None
    body = body.lower()
    body = deEmojify(body)
    body = REPLACE_BY_SPACE_RE.sub(' ', body)
    body = cr_pattern.sub(' ',body)
    l = []
    for token in body.split(" "):
        if(re.search(ver,token)):
            token = token
        elif(re.search(fs,token)):
            l += [dot.sub(' ',token)]
            continue
        l += [token]
    body = ' '.join(token for token in l)
    body = re.sub('\s+',' ',body)             
    body = body.strip()
    return body

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner']) #pos tagging + lammetization
nlp.max_length = 3000000
def pos_tag(corpus):
    corpus = (" ".join([token.lemma_ for token in nlp(corpus) if token.pos_ in ('NOUN','VERB','ADJ','ADV','NUM')]))
    return corpus

In [ ]:
for i in range(len(corpus)):
    corpus[i] = preprocess(corpus[i])
    corpus[i] = pos_tag(corpus[i])
    corpus[i] = REPLACE_BY_NULL_RE.sub('', corpus[i])   #removing \' incase if it didn't get eliminated in lammetization
    corpus[i] = remove_stopwords(corpus[i])             #removing stopwords
    n_st = " "
    corpus[i] = n_st.join(stemmer.stem(token) for token in corpus[i].split()) #stemming

In [ ]:
print(i)

24488


In [ ]:
dataset = [d.split() for d in corpus] #form bigrams and trigrams
bigram = gensim.models.phrases.Phrases(dataset, min_count=10, threshold=20.0)
trigram = gensim.models.phrases.Phrases(bigram[dataset], min_count=10, threshold=20.0)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
from collections import Counter
l1,l2 = [],[]
reg = '\w+\_\w+'        #regex to identify a bigram
rege = '\w+\_\w+\_\w+'  #regex to identify a trigram
for i in range(len(dataset)):
    for token in trigram[bigram[dataset[i]]]:
        if (re.search(reg,token)):
            l1 += [token]
        if (re.search(rege,token)):
            l2 += [token]
Counter1 = Counter(l1)
most_occur = Counter1.most_common(40)    #calculates 40 common bigrams
print(most_occur)
Counter = Counter(l2)
most_occuring = Counter.most_common(40)  #calculates 40 common trigrams
print(most_occuring)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[('main_page_help_geek', 21831), ('org_mail', 21483), ('incorrect_click', 17816), ('comment_incorrect_want_share', 5282), ('inform_topic_discuss', 5262), ('time_complex', 3634), ('strong_recommend_practic', 2549), ('accept_paramet', 1540), ('binari_tree', 1445), ('solv_problem', 1102), ('simpl_solut', 742), ('binari_search', 620), ('naiv_approach', 592), ('auxiliari_space_o', 585), ('prime_factor', 568), ('effici_solut', 562), ('leav_right', 546), ('great_languag_data_analysi', 527), ('primarili_fantast_ecosystem_packag', 527), ('packag_import_analyz_datum', 525), ('list_comprehens', 524), ('problem_solv', 506), ('datum_structur', 503), ('compil_error', 498), ('extra_space', 493), ('machin_learn', 491), ('hash_tabl', 477), ('accept_singl_paramet', 472), ('row_column', 471), ('float_point', 440), ('bad_case', 424), ('leaf_node', 417), ('accept_mandatori_paramet', 410), ('left_right', 382), ('throw_except', 367), ('binari_represent', 367), ('binari_search_tree', 361), ('data_structur', 3

In [ ]:
#retaining 40 most common bigrams and trigrams in the corpus
bigramset = []   
trigramset = []
for i,j in zip(most_occur,most_occuring):
    bigramset.append(i[0])
    trigramset.append(j[0])
reg = '\w+\_\w+'
rege = '\w+\_\w+\_\w+'
REG = re.compile('[\_]')
for i in range(len(dataset)):
    st = ""
    for token in trigram[bigram[dataset[i]]]:
        if re.search(reg,token) and token not in bigramset :
            token = REG.sub(' ',token)
        elif re.search(rege,token) and token not in trigramset:
            token = REG.sub(' ',token)
        st += " "+token
    corpus[i] = st.strip() 

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
from collections import Counter
l = []
for i in range(len(corpus)): 
    l+=(corpus[i].split()) 
Counter1 = Counter(l)
most_occuring = Counter1.most_common(40) #calculates 40 common words in corpus
print(most_occuring)

[('articl', 108339), ('geeksforgeek', 88513), ('link', 79968), ('contribut', 71955), ('use', 64918), ('1', 51088), ('code', 50162), ('close', 40515), ('like', 36477), ('method', 36395), ('improv', 35976), ('edit', 35975), ('2', 35554), ('valu', 33861), ('write', 31948), ('return', 31060), ('exampl', 31021), ('number', 30543), ('output', 28980), ('function', 26350), ('element', 25881), ('appear', 22783), ('array', 22155), ('main_page_help_geek', 21831), ('org_mail', 21483), ('program', 21165), ('button', 18307), ('3', 17912), ('incorrect_click', 17816), ('string', 16568), ('0', 15586), ('object', 13310), ('list', 12562), ('class', 12447), ('time', 12190), ('set', 11812), ('approach', 11589), ('paramet', 11294), ('4', 11266), ('implement', 10825)]


In [ ]:
#removing 40 most common words from the corpus
freq_words = []
for i in most_occuring:
    freq_words.append(i[0])
for i in range(len(corpus)):
    corpus[i] = " ".join(words for words in corpus[i].split() if words not in freq_words)

Pre-processed data

In [ ]:
for i in range(0,11):
  print(corpus[i])
  print('\n')  

techniqu opencv adapt threshold prerequisit simpl threshold opencvin previous post simpl threshold explain differ type threshold techniqu threshold techniqu adapt threshold simpl threshold global threshold remain constant constant threshold win t help case variabl light condit differ area adapt threshold threshold calcul small region lead differ threshold differ region respect chang light cv2 adaptivethreshold syntax adaptivethreshold sourc maxval adaptivemethod blocksiz constant sourc input imag singl channel 8 bit float_point maxval maximum assign adaptivemethod adapt decid threshold calcul threshold mean neighbourhood area constant word mean neighborhood point constant threshold gaussian weight sum neighbourhood constant word weight sum neighborhood point constant thresholdtyp type threshold appli blocksiz size neighborhood calcul threshold constant constant subtract mean weight sum neighbourhood pixel input imag


generat n factor factor task print n ascend order factor problem mai

In [ ]:
#deallocating space for temporary variables
dataset,l1,l2,l = [],[],[],[]
most_occur,most_occuring = [],[] 

K-Means SK-Learn 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.cluster import KMeans 
from sklearn.metrics import silhouette_score 
import math,time 
vec = TfidfVectorizer() 
t0 = time.time()    
vector = vec.fit(corpus) 
X = vector.transform(corpus) 
if X.shape[0] > 5000: 
  k = math.ceil(X.shape[0]/5000) 
else: 
  k = 2 
km = KMeans(n_clusters = k,random_state=42) 
km.fit(X) 
labels = km.labels_ 
print(labels) 
t_km = time.time() - t0 
print(t_km) 

[4 2 4 ... 4 2 4]
418.1813747882843


Mini Batch sklearn

In [ ]:
#collecting mini-batch objects for all 63 
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import math,time
from joblib import parallel_backend 
vec = TfidfVectorizer() 
mk_ob = []
t0 = time.time()
with parallel_backend('threading', n_jobs=2):
    X = vec.fit_transform(corpus)
    if X.shape[0] > 5000:
        k = math.ceil(X.shape[0]/5000)
    else:
        k = 2
    mk = MiniBatchKMeans(n_clusters = k,random_state = 42,batch_size = 5000,max_iter = 500)
    mk.fit(X)
mb_labels = mk.labels_
t_mini_batch = time.time() - t0
print(t_mini_batch)

1.5600266456604004


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/cluster 1.csv" , encoding = "ISO-8859-1")
df

,content,cluster
0,python threshold techniqu opencv adapt thresho...,4
1,generat factor factor task print ascend order ...,2
2,oper question 17 follow differ mean differ con...,4
3,sympi subset size python subset size size symp...,4
4,python pil imagefont truetyp pil python imag l...,4
...,...,...
24484,import hashcod prerequisit equal hashcod javah...,4
24485,queue inbuilt queue perman allow push pop oper...,4
24486,scala field overrid orient languag overrid all...,4
24487,sum minimum possibl sub task sum minimum possi...,2


In [ ]:
df.cluster.value_counts()

4    14725
2     6586
0     1103
1     1074
3     1001
Name: cluster, dtype: int64

In [ ]:
def cos_similarity(textlist):
    tfidf = vec.fit_transform(textlist)
    return (tfidf * tfidf.T).toarray()

In [ ]:
label1=[]
label2=[]
label3=[]
label4=[]
label5=[]
for i in range(len(corpus)):
  if labels[i]==0:
    label1.append(i)
  elif labels[i]==1:
    label2.append(i)
  elif labels[i]==2:
    label3.append(i)
  elif labels[i]==3:
    label4.append(i)
  elif labels[i]==4:
    label5.append(i)
print(label1)
print(len(label1))
print(label2)
print(len(label2))
print(label3)
print(len(label3))
print(label4)
print(len(label4))
print(label5)
print(len(label5))

[5, 10, 12, 69, 117, 122, 190, 206, 233, 246, 255, 270, 304, 311, 318, 404, 445, 461, 476, 496, 515, 518, 562, 586, 659, 690, 779, 833, 874, 880, 897, 905, 909, 914, 929, 950, 968, 1016, 1027, 1029, 1039, 1102, 1129, 1158, 1169, 1172, 1183, 1184, 1192, 1211, 1228, 1239, 1248, 1259, 1272, 1274, 1297, 1326, 1375, 1385, 1433, 1461, 1465, 1519, 1561, 1594, 1615, 1619, 1641, 1668, 1713, 1736, 1790, 1804, 1859, 1947, 1967, 1983, 1984, 1992, 1997, 2010, 2019, 2024, 2032, 2049, 2066, 2071, 2115, 2118, 2127, 2134, 2166, 2203, 2224, 2226, 2228, 2288, 2347, 2370, 2418, 2430, 2449, 2492, 2494, 2518, 2541, 2546, 2576, 2606, 2622, 2646, 2648, 2669, 2681, 2686, 2706, 2736, 2775, 2777, 2832, 2844, 2868, 2886, 2893, 2894, 2897, 2920, 2939, 2948, 2994, 2996, 3015, 3038, 3072, 3202, 3214, 3218, 3221, 3253, 3258, 3403, 3421, 3451, 3461, 3467, 3476, 3498, 3512, 3522, 3654, 3662, 3667, 3694, 3701, 3712, 3729, 3752, 3766, 3782, 3801, 3814, 3823, 3856, 3858, 3953, 3973, 3998, 4012, 4016, 4028, 4043, 4045, 408

Cosine similarity values in a cluster

In [ ]:
clu_str = []
in_dict = {}
j = 0
for i in range(len(labels)):
  if labels[i] == 0:
    in_dict[j] = i
    clu_str += [corpus[i]]
    j += 1
ar = cos_similarity(clu_str)
print(ar)

[[1.         0.09270803 0.10093959 ... 0.04210546 0.01877734 0.13123812]
 [0.09270803 1.         0.26539969 ... 0.02719306 0.04252079 0.13372138]
 [0.10093959 0.26539969 1.         ... 0.05839295 0.04156688 0.08454049]
 ...
 [0.04210546 0.02719306 0.05839295 ... 1.         0.04902489 0.02917496]
 [0.01877734 0.04252079 0.04156688 ... 0.04902489 1.         0.02474727]
 [0.13123812 0.13372138 0.08454049 ... 0.02917496 0.02474727 1.        ]]


**Generating Recommendations**

In [ ]:
for ind in range(0,2):
    related_docs_indices = ar[ind].argsort()[-1:-7:-1] #finding top 6 cosine similarities of that index
    #for j in range(len(related_docs_indices)):
    #  print(j)
    #print(related_docs_indices)
    for i in related_docs_indices:
      #print(in_dict[ind])
      print(ar[ind][i])    
      print(titletoind_dict[in_dict[i]])

1.0
Sum of all natural numbers in range L to R
0.5979694179944263
Sum of all odd natural numbers in range L and R
0.5452725746546961
Sum of all even numbers in range L and R
0.540122236721583
Natural Numbers
0.45458688331235253
Sum of natural numbers using recursion
0.43707513203994797
Find the average of first N natural numbers
1.0
Subsets having Sum between A and B
0.40423095376048684
Sum of subset differences
0.40236745900243415
Find the sum of maximum difference possible from all subset of a given array.
0.40060215594569526
Number of subsets with zero sum
0.37678818900433864
Recursive program to print all subsets with given sum
0.37463454801934976
Maximum size subset with given sum


In [ ]:
data["top1"]=""
data["top2"]=""
data["top3"]=""
data["top4"]=""
data["top5"]=""
data

,title,content,tags,top1,top2,top3,top4,top5
0,Python | Thresholding techniques using OpenCV ...,Prerequisite: Simple Thresholding using OpenC...,"{'Python', 'OpenCV', 'Image-Processing'}",,,,,
1,Generation of n numbers with given set of factors,"Given an array of k numbers factor[], the task...","{'prime-factor', 'Mathematical'}",,,,,
2,C | Operators | Question 17,Which of the following can have different mean...,"{'C', 'C-Operators', 'Operators', 'C Quiz'}",,,,,
3,SymPy | Subset.size() in Python,Subset.size() : size() is a sympy Python libra...,{'Python'},,,,,
4,Python PIL | ImageFont.truetype(),PIL is the Python Imaging Library which provid...,"{'Python', 'Python-pil'}",,,,,
...,...,...,...,...,...,...,...,...
24484,Importance of Hashcode method in Java,Prerequisite: Equals() and hashCode() methods ...,"{'Java', 'java-hashset', 'Java-HashMap'}",,,,,
24485,Ruby | Queue close() function,The close() is an inbuilt function in Ruby cl...,"{'Ruby Collections', 'Ruby'}",,,,,
24486,Scala | Field Overriding,"In any object-oriented programming language, O...","{'Scala', 'Scala-OOPS'}",,,,,
24487,Sum of minimum elements of all possible sub-ar...,"Given an array arr[], the task is to find the ...","{'C++ Programs', 'Data Structures', 'Stack', '...",,,,,


Importing recommendations into dataset

In [ ]:
for ind in range(0,len(label5)):
    related_docs_indices = ar[ind].argsort()[-1:-7:-1] 
   # print(related_docs_indices)#finding top 6 cosine similarities of that index
    for j in range(len(related_docs_indices)):
      if j == 1:
        a=related_docs_indices[j]
        data.loc[in_dict[ind], 'top1'] = titletoind_dict[in_dict[a]] 
      elif j == 2:
        a=related_docs_indices[j]
        data.loc[in_dict[ind], 'top2'] = titletoind_dict[in_dict[a]] 
      elif j == 3:
        a=related_docs_indices[j]
        data.loc[in_dict[ind], 'top3'] = titletoind_dict[in_dict[a]] 
      elif j == 4:
        a=related_docs_indices[j]
        data.loc[in_dict[ind], 'top4'] = titletoind_dict[in_dict[a]] 
      elif j == 5:
        a=related_docs_indices[j]
        data.loc[in_dict[ind], 'top5'] = titletoind_dict[in_dict[a]] 

    
        #print(titletoind_dict[in_dict[a]])
    
      #data.loc[in_dict[ind], 'Top1'] = titletoind_dict[in_dict[ind]]    
data

,title,content,tags,top1,top2,top3,top4,top5
0,Python | Thresholding techniques using OpenCV ...,Prerequisite: Simple Thresholding using OpenC...,"{'Python', 'OpenCV', 'Image-Processing'}",Python | Thresholding techniques using OpenCV ...,Python | Thresholding techniques using OpenCV ...,OpenCV: Segmentation using Thresholding,MATLAB | Converting a Grayscale Image to Binar...,Python | Pandas Series.clip_upper()
1,Generation of n numbers with given set of factors,"Given an array of k numbers factor[], the task...","{'prime-factor', 'Mathematical'}",Python | Count the array elements with factors...,Find number of factors of N when location of i...,Sort on the basis of number of factors using STL,Sort elements on the basis of number of factors,Find four factors of N with maximum product an...
2,C | Operators | Question 17,Which of the following can have different mean...,"{'C', 'C-Operators', 'Operators', 'C Quiz'}",C | Pointer Basics | Question 8,C | Operators | Question 26,C | Misc | Question 7,C | Data Types | Question 9,C | Pointer Basics | Question 17
3,SymPy | Subset.size() in Python,Subset.size() : size() is a sympy Python libra...,{'Python'},Set size() method in Java with Example,List size() method in Java with Examples,ArrayList size() method in Java with Examples,Ruby | Set size() function,SymPy | Polyhedron.size() in Python
4,Python PIL | ImageFont.truetype(),PIL is the Python Imaging Library which provid...,"{'Python', 'Python-pil'}",Python PIL | ImageFont.load_default(),How to set the Font of the Button in C#?,JavaFX | Font Class,How to set the font of the content present in ...,How to set the Font of the MaskedTextBox in C#?
...,...,...,...,...,...,...,...,...
24484,Importance of Hashcode method in Java,Prerequisite: Equals() and hashCode() methods ...,"{'Java', 'java-hashset', 'Java-HashMap'}",Method Class | hashCode() Method in Java,Set hashCode() method in Java with Examples,LocalTime hashCode() method in Java with Examples,How String Hashcode value is calculated?,Float hashCode() method in Java with examples
24485,Ruby | Queue close() function,The close() is an inbuilt function in Ruby cl...,"{'Ruby Collections', 'Ruby'}",Reversing a Queue,queue::push() and queue::pop() in C++ STL,Ruby | Queue pop() function,C# Queue with Examples,Ruby | Queue closed? function
24486,Scala | Field Overriding,"In any object-oriented programming language, O...","{'Scala', 'Scala-OOPS'}",Method Overriding in Scala,C# | Method Overriding,Overriding methods from different packages in ...,override Keyword in C++,Covariant return types in Java
24487,Sum of minimum elements of all possible sub-ar...,"Given an array arr[], the task is to find the ...","{'C++ Programs', 'Data Structures', 'Stack', '...",Sum of maximum elements of all possible sub-ar...,Smallest index such that there are no 0 or 1 t...,Print next greater number of Q queries,Python | Pandas Index.argmin(),Iterative Quick Sort


In [ ]:
data.to_csv('/content/drive/MyDrive/Recommendations.csv')

In [ ]:
#avg cosine similarities of each cluster for a particular board 
import statistics 
for l in range(5): 
    clu_str = []  
    for i in range(len(labels)): 
        if labels[i] == l: 
            clu_str += [corpus[i]] 
    ar = cos_similarity(clu_str) 
    avg = 0 
    for i in range(len(ar)): 
        ind = ar[i].argsort()[-2:-7:-1] 
        avg = avg + (statistics.mean(ar[i][ind][:5])) 
    print(avg/len(ar)) 

0.46573982040127815
0.41799066730306067
0.5171674929673996
0.48341903326098246
0.5787138404083364


In [ ]:
import statistics 
for l in range(5): 
    clu_str = [] 
    for i in range(len(mb_labels)): 
        if mb_labels[i] == l: 
            clu_str += [corpus[i]] 
    ar = cos_similarity(clu_str) 
    avg = 0 
    for i in range(len(ar)): 
        ind = ar[i].argsort()[-2:-7:-1] 
        avg = avg + (statistics.mean(ar[i][ind][:5])) 
    print(avg/len(ar)) 

0.5229362636870544
0.5027681750109002
0.48398418981283814
0.5014293986155416
0.5649332557020926


**Classification algorithm**

In [ ]:
from sklearn.model_selection import train_test_split
print("test shape : ", test.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error,mean_absolute_error, make_scorer,classification_report,confusion_matrix,accuracy_score,roc_auc_score,roc_curve

In [ ]:
tf_idf_vect = TfidfVectorizer(ngram_range=(0,5))
X_train = tf_idf_vect.fit_transform(train.content.values)
X_test = tf_idf_vect.transform(test.content.values)

y_train = train.cluster.values
y_test = test.cluster.values

print("X_train.shape : ", X_train.shape)
print("X_test.shape : ", X_test.shape)
print("y_train.shape : ", y_train.shape)
print("y_test.shape : ", y_test.shape)

X_train.shape :  (17142, 3153757)
X_test.shape :  (7347, 3153757)
y_train.shape :  (17142,)
y_test.shape :  (7347,)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_prediction = knn.predict(X_test)
knn_accuracy = accuracy_score(y_test,knn_prediction)
print("training accuracy Score    : ",knn.score(X_train,y_train))
print("testation accuracy Score : ",knn_accuracy )
print(classification_report(knn_prediction,y_test))

training accuracy Score    :  0.7993232994983083
testation accuracy Score :  0.7211106574111883
              precision    recall  f1-score   support

           0       0.26      0.84      0.39       101
           1       0.19      0.80      0.30        75
           2       0.33      0.79      0.46       813
           3       0.73      0.90      0.81       245
           4       0.97      0.70      0.81      6113

    accuracy                           0.72      7347
   macro avg       0.49      0.81      0.56      7347
weighted avg       0.87      0.72      0.76      7347



Gives cluster for every new data

In [ ]:
## copying two strings from our dataset
text_string=df['content'][5],df['content'][2],df['content'][3]
#text_string='generat n factor factor task print n ascend order factor problem main extens ugli problem creat auxiliari size factor track multipl factor iter minimum increment respect factor minimum equal previous increment avoid duplic print minimum n k auxiliari_space_o k comment_incorrect_want_share inform_topic_discuss','sum minimum possibl sub task sum minimum'
## Applying tfidf to two strings
vector=tf_idf_vect.transform(text_string)
## Getting the prediction
knn.predict(vector)

array([0, 4, 4])

FAISS Indexing method

In [ ]:
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
X = vec.fit_transform(corpus) #can take whole corpus
d = X.shape[1]                #fit the corpus to tf-idf vector as X
n = 48                        #no.of clusters
quantiser = faiss.IndexFlatL2(d)  
index = faiss.IndexIVFFlat(quantiser, d, n, faiss.METRIC_L2) #creating a index
index.train(X)                                     
index.add(X)
print(index.ntotal) 

89059


In [ ]:
#find index of a conversation in X 
ind = indtoid_dict['11064351']

In [ ]:
D, I = index.search(X[ind:ind+1], 100)     # gives 100 nearest neighbors for the conversation 
print(list(I))

[array([ 9468, 20682,  3476, 32489, 73460, 57825, 24801, 24581,   184,
       55534,  4420,  1086, 34202,  8374,  9693, 39732, 61888, 20844,
       54138, 11990, 70427, 16803, 21386, 20569,  5157, 42318, 21555,
       30822, 41430, 41486,  5950, 19896, 29087, 53709, 64616, 66449,
       10456, 33073, 79315, 27744, 12581, 54989, 77915, 53930, 18743,
       76719,  7054, 25351, 57643, 10347, 40087,  6275, 59231,  1838,
       22564, 59949, 83903, 32017, 35638, 28044, 28273, 51626, 32526,
       80665,  2791, 27591, 63626, 54948, 53659, 45710, 82132, 13128,
       84825, 24906, 52733, 12604, 74880, 55397, 86565, 87370, 24947,
       50881, 20591, 86655, 66826, 52600, 79676, 34163, 15461, 80033,
         233, 13674, 28642, 59573,  5349, 12545, 50724, 86077, 37641,
       11684])]


In [ ]:
def cos_similarity(textlist):
    tfidf = vec.fit_transform(textlist)
    return (tfidf * tfidf.T).toarray()

In [ ]:
#find cosine similarities for that conversation and its neighbors
in_dict = {}
str_list = []
j = 0
for i in list(I)[0]:
    str_list += [corpus[i]]
    in_dict[j] = i
    j+=1
ar = cos_similarity(str_list)
related_docs_indices = ar[0].argsort()[:-7:-1]
related_docs_indices
for i in related_docs_indices:
    print(ar[0][i],indtoid_dict[in_dict[i]])

NameError: ignored

FAISS KMeans Clustering

In [ ]:
import faiss
k = 48 #no.of clusters
niter = 200
verbose = True
d = X.shape[1] #fit corpus to tf-idf vector as X
kmeans = faiss.Kmeans(d, k, niter=niter, verbose=verbose)
kmeans.train(X)

7391.58837890625


In [ ]:
D, I = kmeans.index.search(X, 1) #assigning cluster labels to corpus

In [ ]:
ind = indtoid_dict['11064351']
I[ind] #gives cluster label for that conversation

array([43])

In [ ]:
l = []
in_dict = {}
k = 0
id_list = list(indtoid_dict.keys())
for i,j in zip(I,range(len(id_list))):
    if i[0] == I[ind][0]:
        l += [corpus[indtoid_dict[j]]]
        in_dict[k] = j
        k += 1
ar = cos_similarity(l)
key_list = list(in_dict.keys()) 
val_list = list(in_dict.values())  
idx = key_list[val_list.index('11064351')] #finding index of '11064351' in cosine similarity matrix
related_docs_indices = ar[idx].argsort()[-1:-7:-1]
for i in related_docs_indices:
    print(ar[idx][i],in_dict[i])

NameError: ignored

Birch

In [ ]:
from sklearn.cluster import Birch
brc = Birch(branching_factor=100, n_clusters= None, threshold=0.82) #gives 43 clusters (change threshold to change no.of clusters formed)
brc.fit_predict(X)

array([ 0,  0,  0, ...,  0, 31,  0])

In [ ]:
labels = brc.labels_
for j in range(43):
    c = 0
    for i in range(len(corpus)):
        if labels[i] == j:
            c+=1
    print("cluster ",j," :",c)

cluster  0  : 62967
cluster  1  : 467
cluster  2  : 70
cluster  3  : 1
cluster  4  : 91
cluster  5  : 283
cluster  6  : 449
cluster  7  : 1044
cluster  8  : 306
cluster  9  : 1804
cluster  10  : 1
cluster  11  : 1082
cluster  12  : 1503
cluster  13  : 455
cluster  14  : 1175
cluster  15  : 1736
cluster  16  : 618
cluster  17  : 1
cluster  18  : 234
cluster  19  : 1
cluster  20  : 1706
cluster  21  : 1
cluster  22  : 473
cluster  23  : 347
cluster  24  : 869
cluster  25  : 1
cluster  26  : 1172
cluster  27  : 783
cluster  28  : 2
cluster  29  : 1233
cluster  30  : 2418
cluster  31  : 1448
cluster  32  : 215
cluster  33  : 4
cluster  34  : 1548
cluster  35  : 1
cluster  36  : 2187
cluster  37  : 1
cluster  38  : 37
cluster  39  : 2
cluster  40  : 1
cluster  41  : 289
cluster  42  : 474


DBSCAN

In [ ]:
#needs more memory couldn't execute got crashed
from sklearn.cluster import DBSCAN 
db = DBSCAN(min_samples= 20,eps = 15,n_jobs = 2)
db.fit(X)